In [11]:
import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import os
import glob
from pathlib import Path
import SimpleITK as sitk # para calcular rangos
import numpy as np
import io
from PIL import Image
import random
import sys
import pandas as pd
import math
import pandas as pd
import sklearn
import sklearn.metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [12]:
sys.path.append("/Users/hugom/PET-IA/src")

In [13]:
from datasets import ADNIDataset, FleniMyriamDataset, BaseDataset
from transforms import TransformGridImage, ToLabelOutputConfigurable
from train_lib import train_model, set_parameter_requires_grad, initialize_model
from util import collectAllData
import torch
import torchvision
from torchvision import transforms, utils, models, datasets

In [14]:
config = {
  "last_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_3classes_0_epoch21.pth",
    "num_classes": 3
  },
  "sev_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_3classes_0_epoch31.pth",
    "num_classes": 3
  },
  "visit_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_3classes_0_epoch10.pth",
    "num_classes": 3
  },
  "last_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_2classes_0_epoch18.pth",
    "num_classes": 2
  },
  "sev_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_2classes_0_epoch16.pth",
    "num_classes": 2
  },
  "visit_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_2classes_0_epoch5.pth",
    "num_classes": 2
  }
}


In [35]:
def executeModel(dataloader, weights, num_classes):
    featureExtract = False
    dropoutRate = 0.6
    auxEnabled = True
    usePretrained = False
    model_ft, input_size = initialize_model('inception', num_classes, featureExtract, dropoutRate, auxEnabled, use_pretrained=usePretrained)
    device = torch.device(deviceName if torch.cuda.is_available() else "cpu")
    model_state_dict = torch.load(weights, map_location=device)
    model_ft.load_state_dict(model_state_dict)

    model_ft.to(device)
    
    testY, predY = collectAllData(dataloader, model_ft, device, num_classes)
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(testY)
    print(preds)
    
    print(f"Accuracy: {acc}")
    print(f"B acc: {bacc}")
    if num_classes == 2:
        f1 = sklearn.metrics.f1_score(testY, preds)
        print(f"F1: {f1}")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    
    print("Confusion matrix:")
    print(matrix)
    
    if (num_classes == 2):
        lr_auc = roc_auc_score(testY, torch.from_numpy(predY[:, 1]), average = None)
        print(f"AUC ROC: {lr_auc}")
        return
    
    print("------------")
    print("Merging 2 with 0")
    
    predY[:, 0] = predY[:, 0] + predY[:, 2]
    predY = predY[:, 0:2]
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    f1 = sklearn.metrics.f1_score(testY, preds)
    
    print(f"Accuracy merged: {acc}")
    print(f"B acc merged: {bacc}")
    print(f"F1 merged: {f1}")
    
    print("Confusion matrix merged:")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    print(matrix)
    
    lr_auc = roc_auc_score(testY, torch.from_numpy(predY[:, 1]), average = None)
    print(f"AUC ROC: {lr_auc}")

In [36]:
csvFile = '/Users/hugom/PET-IA/Sets/fleni-myriam-curated.csv'
imagesFolder = '/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3'
mean = 3383.638427734375
std = 7348.981426020888
means = [mean, mean, mean]
stds = [std, std, std]
transforms = torchvision.transforms.Compose([
    TransformGridImage(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(means, stds)
])
dicti = {
    "AD": 1,
    "non-AD": 0
}
dataset = FleniMyriamDataset('fleni', '/Users/hugom/PET-IA/Sets/fleni-myriam-curated.csv', '/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3', transform = transforms)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [37]:
for key in config:
    print(f"Ejecutando {key}")
    executeModel(dataloader, **config[key])
    print("#################")

Ejecutando last_3_classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0
 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 0
 0 1 0 0 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1]
Accuracy: 0.5
B acc: 0.4625
Confusion matrix:
[[11 29]
 [21 39]]
------------
Merging 2 with 0
Accuracy merged: 0.49
B acc merged: 0.45416666666666666
F1 merged: 0.5984251968503937
Confusion matrix merged:
[[11 29]
 [22 38]]
AUC ROC: 0.5070833333333333
#################
Ejecutando sev_3_classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048


...

[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[1 2 1 1 2 1 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 1 2 2 2 2 1 1 1 1 2 2 1 1 1 2 2
 1 2 1 2 1 2 2 2 2 2 0 1 2 2 1 1 2 1 2 1 2 2 2 0 0 2 1 1 2 2 2 2 2 1 2 1 0
 2 2 2 2 1 2 1 2 2 2 0 1 2 2 2 2 2 2 2 1 1 2 2 2 2 2]
Accuracy: 0.18
B acc: 0.15416666666666667
Confusion matrix:
[[ 1 14 25]
 [ 4 17 39]
 [ 0  0  0]]
------------
Merging 2 with 0
Accuracy merged: 0.43
B acc merged: 0.4958333333333333
F1 merged: 0.2597402597402597
Confusion matrix merged:
[[33  7]
 [50 10]]
AUC ROC: 0.46874999999999994
#################
Ejecutando last_2classes
num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0
 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1]
Accuracy: 0.57
B acc: 0.5083333333333333
F1: 0.6950354609929078
Confusion matrix:
[[ 8 32]
 [11 49]]
AUC ROC: 0.5064583333333333
#################
Ejecutando visit_2classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.4
B acc: 0.5
F1: 0.0
Confusion matrix:
[[40  0]
 [60  0]]
AUC ROC: 0.5191666666666667
#################


# Nuevos

In [38]:
configNew = {
  "last_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxlast_2classes_CN,AD_0_epoch42.pth",
    "num_classes": 2
  },
  "sev_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxmost_severe_2classes_CN,AD_0_epoch12.pth",
    "num_classes": 2
  },
  "visit_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxvisit_2classes_CN,AD_0_epoch19.pth",
    "num_classes": 2
  },
  "last_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxlast_2classes_MCI,AD_0_epoch45.pth",
    "num_classes": 2
  },
  "sev_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxmost_severe_2classes_MCI,AD_0_epoch17.pth",
    "num_classes": 2
  },
  "visit_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxvisit_2classes_MCI,AD_0_epoch1.pth",
    "num_classes": 2
  }
}


In [39]:
csvFile = '/Users/hugom/PET-IA/Sets/fleni-myriam-curated.csv'
imagesFolder = '/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3'
means = [mean, mean, mean]
stds = [std, std, std]
ebrainsTransforms = torchvision.transforms.Compose([
    TransformGridImage(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(means, stds)
])
dicti = {
    "AD": 1,
    "non-AD": 0
}
dataset = FleniMyriamDataset('fleni', '/Users/hugom/PET-IA/Sets/fleni-myriam-curated.csv', '/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3', transform = transforms)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [40]:
for key in configNew:
    print(f"Ejecutando {key}")
    executeModel(dataloader, **configNew[key])
    print("#################")

Ejecutando last_cn_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[1 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1
 1 0 1 1 1 0 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0 0 1 1]
Accuracy: 0.51
B acc: 0.46249999999999997
F1: 0.6315789473684211
Confusion matrix:
[[ 9 31]
 [18 42]]
AUC ROC: 0.50125
#################
Ejecutando sev_cn_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Accuracy: 0.6
B acc: 0.5
F1: 0.7499999999999999
Confusion matrix:
[[ 0 40]
 [ 0 60]]
AUC ROC: 0.5533333333333333
#################
Ejecutando visit_cn_vs_ad


...

num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1
 0 1 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0]
Accuracy: 0.43
B acc: 0.475
F1: 0.3448275862068966
Confusion matrix:
[[28 12]
 [45 15]]
AUC ROC: 0.49833333333333335
#################
Ejecutando sev_mci_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0]
Accuracy: 0.51
B acc: 0.5625
F1: 0.4235294117647059
Confusion matrix:
[[33  7]
 [42 18]]
AUC ROC: 0.48083333333333333
#################
Ejecutando visit_mci_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.41
B acc: 0.5083333333333333
F1: 0.03278688524590164
Confusion matrix:
[[40  0]
 [59  1]]
AUC ROC: 0.5175
#################
